In [3]:
import pandas as pd
import json
import requests 
from python_secrets import *

In [4]:
df_main = pd.read_csv('incident2.csv')

In [5]:
tx_hash_list = [x for x in df_main[' user_tx'].to_list() if pd.notnull(x)]

# Infuria

In [6]:
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionByHash",
  "params": [tx_hash_list[0]],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dct = response.json()['result']
dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
df_infuria = pd.DataFrame(dct, index=[0])

for tx_hash in tx_hash_list[1:]:
  payload = json.dumps({
    "jsonrpc": "2.0",
    "method": "eth_getTransactionByHash",
    "params": [tx_hash],
    "id": 1
  })
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  if response.ok:
    dct = response.json()['result']
    dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct, index=[0])
    df_infuria = pd.concat([df_infuria, df_temp])
    
  else: 
    print(f"error code {response.status_code} for transaction {tx_hash}")


In [7]:
#merge infuria to main
df_main = df_main.merge(df_infuria, left_on = ' user_tx', right_on = 'hash', how ='outer')

In [8]:
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity'],
      dtype='object')

# Etherscan 

If we always have the dollar values from the tenderly api, we actually don't need to call ethersacan for the time stamps

In [356]:
block_number_list = list(set([x for x in df_main['block_number'].to_list() if pd.notnull(x)]))

In [357]:
url_eth = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number_list[0]}&apikey={eth_scan_api_key}"

response_eth = requests.request("POST", url_eth)

dct_eth = response_eth.json()['result']
dct_eth = {k: None if not v else v for k, v in dct_eth.items()} # making sure none of the values are empty
df_eth = pd.DataFrame(dct_eth, index=[0])

for block in block_number_list[1:]:
  url_temp = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block}&apikey={eth_scan_api_key}"
  response_temp = requests.request("POST", url_temp)

  if response_temp.ok:
    dct_temp = response_temp.json()['result']
    dct_temp = {k: None if not v else v for k, v in dct_temp.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct_temp, index=[0])
    df_eth = pd.concat([df_eth, df_temp])
    
  else: 
    print(f"error code {response_temp.status_code} for block {block}")

KeyboardInterrupt: 

In [ ]:
df_eth['blockNumber'] = df_eth['blockNumber'].astype(int)

In [ ]:
#merge eth to main
df_main = df_main.merge(df_eth, left_on = 'block_number', right_on = 'blockNumber', how ='outer')

# Tenderly

In [9]:
headers = {
    'X-Access-Key': f'{tenderly_access_token}',
    'content-type': 'application/json',
}

columns = ['tx_hash', 'index', 'type', 'raw_amount', 'dollar_value', 'token_contract_address', 'token_name', 'token_dollar_value']
df_results = pd.DataFrame(columns = columns)
tx_hash_problem_list = []

for index, row in df_main.iterrows():
    tx_index_list = [0]
    tx_index_list.append(int(row['transactionIndex'], 0))
    for tx_index in tx_index_list:
        json_data = {
        'network_id': int(row['chainId'], 0),
        'from': row['from'],
        'to': row['to'],
        'input': row['input'],
        'block_number': row['block_number'],
        'transaction_index': tx_index,
        'simulation_type': 'quick',
        'gas': int(row['gas'], 0),
        'value': int(row['value'], 0),
        'gas_price': int(row['gasPrice'], 0)
   
    }
        response = requests.post(
        'https://api.tenderly.co/api/v1/account/aurelie/project/cowswap/simulate',
        headers=headers,
        json=json_data,
        )
        
        try:
            for data in response.json()['transaction']['transaction_info']['asset_changes']:
                tx_type = data['type']
                tx_raw_amount = data['raw_amount']
                tx_dollar_value = data['dollar_value']

                try:
                    contract_address = data['token_info']['contract_address']
                except:
                    contract_address = 'None'

                try:
                    token_name = data['token_info']['name']
                except:
                    token_name = 'None'
                try:
                    token_dollar_value = data['token_info']['dollar_value']
                except:
                    token_dollar_value = 'None'

                new_row = {
                    'tx_hash' : row['hash'],
                    'index' : tx_index, 
                    'type': tx_type, 
                    'raw_amount': tx_raw_amount, 
                    'dollar_value' : tx_dollar_value, 
                    'token_contract_address': contract_address, 
                    'token_name': token_name, 
                    'token_dollar_value': token_dollar_value
                    }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except:
            example = response.json()
            tx_hash_problem = df_main[df_main['input'] == row['input']]['hash']
            tx_hash_problem_list.append(tx_hash_problem)
            


In [10]:
with open("Output.json", "w") as text_file:
    text_file.write(str(example))


In [136]:
grouped = df_results.groupby('tx_hash')
df_results_good = grouped.filter(lambda x: x['index'].nunique() == 2)

In [137]:
# number of transactions for which this approach worked
df_results_good.tx_hash.nunique()

28

In [138]:
df_results_good = df_results_good.groupby(['tx_hash', 'index', 'token_contract_address'], as_index=False).first()
df_results_good['dollar_value'] = df_results_good['dollar_value'].astype(float)
df_results_good['asset_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['dollar_value'].diff()

In [139]:
#total potential loss in dollars
df_results_good.asset_diff.sum()

-5544.377747019963

In [140]:
df_results_good

,tx_hash,index,token_contract_address,type,raw_amount,dollar_value,token_name,token_dollar_value,asset_diff
0,0x012778bb6330737bed53ca488e582500498d81e1db22...,0,0x4fe8d4775b7cb2546b9ee86182081cdf8f77b053,Transfer,16544534111992633092354532893,650.299432,KAIJUNO8,0.000000039305998456029556,NaN
1,0x012778bb6330737bed53ca488e582500498d81e1db22...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Mint,200000000000000000,713.127979,WETH,3565.639892578125,NaN
2,0x012778bb6330737bed53ca488e582500498d81e1db22...,0,None,Transfer,200000000000000000,713.147998,Ethereum,3565.739990234375,NaN
3,0x012778bb6330737bed53ca488e582500498d81e1db22...,2,0x4fe8d4775b7cb2546b9ee86182081cdf8f77b053,Transfer,11103714626240328921002570571,436.442590,KAIJUNO8,0.000000039305998456029556,-213.856842
4,0x012778bb6330737bed53ca488e582500498d81e1db22...,2,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Mint,200000000000000000,713.127979,WETH,3565.639892578125,0.000000
...,...,...,...,...,...,...,...,...,...
161,0xf8196d4b1341fb7700603dd0abfebc5ee859b279eb9c...,0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Mint,3000000000000000000,10696.710205,WETH,3565.570068359375,NaN
162,0xf8196d4b1341fb7700603dd0abfebc5ee859b279eb9c...,0,None,Transfer,3000000000000000000,10694.849854,Ethereum,3564.949951171875,NaN
163,0xf8196d4b1341fb7700603dd0abfebc5ee859b279eb9c...,4,0x21e5c85a5b1f38bddde68307af77e38f747cd530,Transfer,1547438874490587247647,2048.809102,Doggensnout Skeptic,0.0000000013240000207304092,-10.243635
164,0xf8196d4b1341fb7700603dd0abfebc5ee859b279eb9c...,4,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Mint,3000000000000000000,10696.710205,WETH,3565.570068359375,0.000000


In [141]:
#df_results_good = df_results_good.dropna(subset=['asset_diff'])[['tx_hash', 'asset_diff']]
#df_results_good.to_csv('preliminary_results.csv')

## Calculate the dollar value of the potential loss for the coins that don't have a dollar value
The logic here is that there is WETH in all the transactions, so we will get the value of the unknown coin based on the WETH value

In [142]:
#first if token name is WETH, then the raw_amount need tos be converted. Every transaction has WETH so I will base the analysis on that 
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good.loc[df_results_good['token_name'] == 'WETH', 'raw_amount'] /= 1e+18

In [147]:
def calculate_dollar_value(row, df):
    if pd.isna(row['dollar_value']):
        # Filter the DataFrame to get the subset with the same 'tx_hash' and 'index' values
        subset = df[(df['tx_hash'] == row['tx_hash']) & (df['index'] == row['index'])]

        if not subset.empty:
            # Calculate dollar value based on 'WETH' value if the subset is not empty
            weth_row = subset[subset['token_name'] == 'WETH']
            if not weth_row.empty:
                weth_raw_amount = float(weth_row['raw_amount'].values[0])
                weth_dollar_value = weth_row['dollar_value'].values[0]
                unknown_raw_amount = float(row['raw_amount'])
                return (weth_raw_amount * weth_dollar_value) / unknown_raw_amount

    # If token_name is not NaN or if the subset is empty, return the original dollar_value
    return row['dollar_value']


In [148]:

# Apply the function to each row
# Iterate over each row in the DataFrame
for index, row in df_results_good.iterrows():
    # Call the calculate_dollar_value function and pass the current row and the DataFrame
    calculated_value = calculate_dollar_value(row, df_results_good)
    
    # Update the 'dollar_value' column with the calculated value
    df_results_good.at[index, 'dollar_value'] = calculated_value


0.4
1426.25595703125
8.492115069140417e+20
0.4
1426.25595703125
8.074525318605014e+20
0.2
713.114013671875
1.4506208337094267e+23
0.2
713.127978515625
2.1032322466908208e+23
0.46855135301681916
1670.6526798060574
218774635948809.0
0.46855135301681916
1670.6526798060574
325968215648902.0
4.45390495201369
15880.71018421761
1.25e+22
4.607552580192021
16428.87328708394
1.25e+22
0.9869972201079622
3519.20774557086
466225481228063.0
0.9869942303024463
3519.197085209802
739617670956575.0
0.6674706864737044
2379.9135011979256
1.4301263248671973e+23
0.6674706864737044
2379.9135011979256
1.603558020058867e+23
1.0
3565.570068359375
4.187961881938826e+21
1.0
3565.570068359375
5.456180989865073e+21
1.8256670581965693
6509.543817495401
1e+19
1.8648905653276575
6649.397980498089
1e+19
0.1
356.5570068359375
6.983662452066881e+22
0.1
356.5570068359375
1.0126294179802136e+23


In [149]:
df_results_good = df_results_good.sort_values(by = ['tx_hash', 'index'], ascending = [True, False])

In [150]:
# FYI if token_contract_address == 'None', then it's always Ethereum
df_results_good['raw_amount'] = df_results_good['raw_amount'].astype(float)
df_results_good['raw_amount_diff'] = df_results_good.groupby(['tx_hash', 'token_contract_address'])['raw_amount'].transform(lambda x: x[::-1].diff())


In [151]:
df_results_good['final_diff'] = df_results_good['raw_amount_diff'] * df_results_good['dollar_value']

In [162]:
#total potential loss in dollars.
#prioritize the difference if it was returned by api, otherwise use our way of calculating
df_results_good[df_results_good.asset_diff.isna()].final_diff.sum() + df_results_good.asset_diff.sum()

-9289.272417443923

In [164]:
df_results_good.to_csv('0204_preliminary_results.csv')